In [1]:
import numpy as np
import pims
import nd2
import xarray
import dask.array as da

In [ ]:
def readND2(filename):
    """
    
    """
    sequence = nd2.imread(filename, xarray = True, dask = True)
    frameT = sequence.metadata['experiment'][0].parameters.periodDiff.avg #this is the time per frame in ms
    scale = sequence.X[1] #this is the microns per pixel
    sequence['T'] = np.arange(len(sequence['T']))*frameT
    sequence.attrs = {'xyScale': float(scale),
                      'tScale': frameT}
    return(sequence)

In [ ]:
seq = readND2('data/A1_s3001.nd2')
seq

In [ ]:
def readLIF(filename):
    """
    
    """
    lifSeq = pims.Bioformats(filename, read_mode = 'jpype')
    xscale = lifSeq.metadata.PixelsPhysicalSizeX(0)
    tscale = lifSeq.metadata.PlaneDeltaT(0,1)
    lifXCoords = np.arange(0., lifSeq.shape[1]*xscale, xscale)
    lifYCoords = np.arange(0., lifSeq.shape[2]*xscale, xscale)
    lifTCoords = np.arange(0., lifSeq.shape[0]*tscale, tscale)
    ds = da.from_array(lifSeq)
    sequence = xarray.DataArray(data = ds, dims = ["T", "Y", "X"], coords = (lifTCoords, lifYCoords, lifXCoords), attrs = dict(xyScale = xscale, tScale = tscale))
    return sequence

In [ ]:
readLIF('data/21-03-31_ddm.lif')

In [3]:
pims.bioformats.download_jar(version='6.5')

'/home/tud258930/anaconda3/lib/python3.8/site-packages/pims/loci_tools.jar'

In [2]:
lifSeq = pims.Bioformats('../data/21-03-31_ddm.lif', read_mode = 'jpype')

In [15]:
lifSeq.metadata.ImageCount() #gives the number of series in the .lif file
readAll = ''
if lifSeq != 1:
    while readAll != 'y' and readAll != 'n':
        readAll = input('Do you want to run the analysis on the entire series? (y/n)')
        if readAll != 'y' and readAll != 'n':
            print('Please type y or n')
        else:
            break

Do you want to run the analysis on the entire series? (y/n)a
Please type y or n
a
Do you want to run the analysis on the entire series? (y/n)y


In [ ]:
def readTIF(filename, xscale = 1., tscale = 1.):
    """
    
    """
    tifSeq = pims.Bioformats(filename, read_mode = 'jpype')
    try:
        xscale = tifSeq.metadata.PixelsPhysicalSizeX(0)
    except:
        xscale = xs
    try:
        tscale = tifSeq.metadata.PixelsTimeIncrement(0)*1000.
    except:
        tscale = ts
    tifXCoords = np.arange(0., tifSeq.shape[1]*xscale, xscale)
    tifYCoords = np.arange(0., tifSeq.shape[2]*xscale, xscale)
    tifTCoords = np.arange(0., tifSeq.shape[0]*tscale, tscale)
    ds = da.from_array(tifSeq)
    sequence = xarray.DataArray(data = ds, dims = ["T", "Y", "X"], coords = (tifTCoords, tifYCoords, tifXCoords), attrs = dict(xyScale = xscale, tScale = tscale))
    return sequence

In [ ]:
readTIF('data/A1_s3001.tif')

In [ ]:
def read_file(filename, xs = None, ts = None):
    """
    """
    supported = ['.lif', '.nd2', '.tif', '.tiff']
    if filename[:-4] not in supported:
        print('Not a supported image format')
        return None
    if filename[:-4] == '.lif'
        return readLIF(filename)
    elif filename[:-4] == 'nd2'
        return readND2(filename)
    else:
        return readTIF(filename, xscale = xs, tscale = ts)

In [ ]:
readfile('x.png')